In [ ]:
%cd ..
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
import imageio

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

# Load an example image

In [ ]:
examples = ["kate", "library", "vase", "vase2"]
example_id = 2
data_path = os.path.join("notebooks", "data", "inpainting")
image = imageio.imread(os.path.join(data_path, f"{examples[example_id]}.png"))
mask = imageio.imread(os.path.join(data_path, f"{examples[example_id]}_mask.png"))

image = datasets.from_255_to_1_range(datasets.to_float32(tf.constant(image)))[None, ...]
mask = tf.broadcast_to(
    datasets.from_255_to_1_range(datasets.to_float32(tf.constant(mask)))[
        None, ..., None
    ],
    shape=tf.shape(image),
)

image_masked = mask * image


## Skimage for comparison

In [ ]:
import skimage.restoration

inpainted_skimage = skimage.restoration.inpaint_biharmonic(
    image[0].numpy(),
    tf.cast(mask[0, ..., 0] == False, tf.uint8).numpy(),
    multichannel=True,
)[None, ...]
inpainted_border = dppp.inpaint_border(image_masked, mask)


## Inpaint using DMSP

In [ ]:
# Load the denoiser
denoiser, (denoiser_min_stddev, denoiser_max_stddev) = dppp.load_denoiser(
    "models/drugan+_0.0-0.2.h5"
)

# Define the noise stddev used for the stochastic evaluation of the prior
if denoiser_min_stddev <= 0.1 <= denoiser_max_stddev:
    denoiser_stddev = 0.1
else:
    denoiser_stddev = denoiser_max_stddev

# Callbacks: Print the PSNR every 20th step
callbacks = [dppp.callback_print_psnr("psnr", 20, image)]

reconstructed = dppp.dmsp_inpaint(
    degraded=image_masked,
    mask=mask,
    denoiser=denoiser,
    denoiser_stddev=denoiser_stddev,
    callbacks=callbacks,
)


In [ ]:
visualize.draw_images(
    [image_masked[0], inpainted_skimage[0], inpainted_border[0], reconstructed[0]]
)
